In [1]:
import pandas as pd
from datetime import date
pd.set_option('display.max_columns', None) #настройка, чтобы видеть все колонки

In [2]:
obs_full = pd.read_csv('data/observations-285432.csv') #создаём новый огромный датафрейм из csv
obs = obs_full[['id','user_login','created_at','quality_grade','scientific_name','common_name']].copy() #создаём новый маленький датафрейм с нужными столбцами
obs['created_at'] = pd.to_datetime(obs['created_at']).dt.date #конвертируем столбец с датами наблюдений в тип datatime
finish_date = max(obs['created_at'])
# finish_date = date(2019,5,30)
# start_date = min(obs['created_at'])
start_date = date(2022,10,2)
print(start_date, finish_date)

2022-10-02 2022-12-23


In [3]:
def prepare_stat(df, date_begin):

    #prepare_stat нужна чтобы создать список "лучших" 
    #наблюдателей с большим количеством наблюдений

    stat = pd.DataFrame()                                                               #создаём новый датафрейм
    counts = df[df['created_at']<=date_begin].loc[:,'user_login'].value_counts()        #подсчитываем количество всех наблюдений у каждого пользователя с датой предшествующей или равной date_begin
    stat['user'] = counts.index                                                         #создаём колонку с именами пользователей
    stat['obs_count'] = counts.to_list()                                                #создаём колонку с кол-вом всех наблюдений
    research = df[(df['quality_grade']=='research')&(df['created_at']<=date_begin)].loc[:,'user_login'].value_counts()         #подсчитываем кол-во наблюдений со статусом research у каждого пользователя
    stat = stat.join(research,on='user', how='left')                                    #создаём колонку с кол-вом наблюдений research, объединяя список всех наблюдений со списком наблюдений research
    stat.rename(columns={'user':'user',                                                 #просто нужно переименовать один столбец из-за особенности join
                        'obs_count':'obs_count',
                        'user_login':'obs_res_count'},
                         inplace=True)
    stat.fillna(0,inplace=True)                                                         #заполняем нулями значения NaN (для тех у кого нет ни одного research)
    stat['obs_res_count'] = stat['obs_res_count'].astype('Int64')                       #меняем "х.0" на "x"
    stat.sort_values(by=['obs_count','obs_res_count','user'],ascending=False,inplace=True)
    stat.insert(1,'position','')
    stat.insert(2,'pos_cool','')
    stat['position'] = range(1,counts.shape[0]+1)                                       #создаём колонку с позицией, взяв кол-во пользователей из shape
    stat['pos_cool'] = stat[['obs_count','obs_res_count']].ne(stat[['obs_count','obs_res_count']].shift()).any(axis=1).cumsum()
    
    return stat

# prepare_stat(obs,start_date)
# prepare_stat(obs,finish_date)

In [4]:
def prepare_changes(df, start_date, finish_date):
    start_obs = prepare_stat(df, start_date)
    finish_obs = prepare_stat(df, finish_date)
    stat = finish_obs.merge(right=start_obs,how='left',left_on='user', right_on='user')   
    stat_changes = pd.DataFrame()
    stat_changes['position'] = stat['position_x']
    stat_changes['pos_cool'] = stat['pos_cool_x']
    stat_changes['pos_cool_shift'] = stat['pos_cool_y'] - stat['pos_cool_x']
    stat_changes['user'] = stat['user']
    stat_changes['obs_count'] = stat['obs_count_x']
    stat_changes['obs_count_shift'] = stat['obs_count_x'] - stat['obs_count_y']
    stat_changes['obs_res_count'] = stat['obs_res_count_x']
    stat_changes['obs_res_count_shift'] = stat['obs_res_count_x'] - stat['obs_res_count_y'] 
    stat_changes['pos_cool_shift'] = stat_changes['pos_cool_shift'].apply(lambda x: ('+' + str(x)) if x>0 else '')
    stat_changes['obs_count_shift'] = stat_changes['obs_count_shift'].apply(lambda x: ('+' + str(x)) if x>0 else '')
    stat_changes['obs_res_count_shift'] = stat_changes['obs_res_count_shift'].apply(lambda x: ('+' + str(x)) if x>0 else '')
    stat_changes.sort_values(by=['obs_count','obs_res_count'],ascending=False,inplace=True)
    return stat_changes

In [5]:
changes = prepare_changes(obs, start_date, finish_date)
changes = changes[((changes['pos_cool'] == 11).cumsum() != 1)]
changes

,position,pos_cool,pos_cool_shift,user,obs_count,obs_count_shift,obs_res_count,obs_res_count_shift
0,1,1,,budetinetakoe,287,+165,192,+102
1,2,2,+7,anastasiiamerkulova,222,+202,160,+143
2,3,3,,elenasuslova,93,,88,
3,4,4,,mitrula_paludosa,62,,55,
4,5,5,,daria_kalinina,41,,37,
5,6,6,,taimyr,37,,37,
6,7,7,,kirill_kuptsov,37,,25,
7,8,8,,bio_4you,30,,23,
8,9,9,,elizaveta_boldinova,24,,24,
9,10,10,,naturalist40722,14,,13,


In [6]:
finish_observations = prepare_stat(obs, finish_date)
start_observations = prepare_stat(obs, start_date)

In [7]:
start_observations

,user,position,pos_cool,obs_count,obs_res_count
0,budetinetakoe,1,1,122,90
1,elenasuslova,2,2,93,88
2,mitrula_paludosa,3,3,62,55
3,daria_kalinina,4,4,41,37
4,taimyr,5,5,37,37
5,kirill_kuptsov,6,6,37,25
6,bio_4you,7,7,30,23
7,elizaveta_boldinova,8,8,24,24
8,anastasiiamerkulova,9,9,20,17
9,naturalist40722,10,10,14,13


In [ ]:
finish_observations

In [ ]:
stat = finish_observations.merge(right=start_observations,how='left',left_on='user', right_on='user')   

In [ ]:
stat

In [ ]:
del(stat)